<a href="https://colab.research.google.com/github/GeorgeTelles/detector_rompimento_topos_fundos/blob/main/detector_rompimento_topos_fundos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://postimg.cc/FYtnXpJw">
  <img src="https://i.postimg.cc/t470PmRR/logo-sem-fundo-01.png" alt="logo" width="300">
</a>

---
# **Detector de rompimento de topos e fundos**

#### O projeto é uma ferramenta automatizada que analisa ativos financeiros para identificar e relatar rompimentos de topos e fundos. O sistema monitora vários ativos, detecta quando estes atravessam níveis críticos e gera relatórios sobre quais ativos estão atualmente rompendo topos ou fundos. Ideal para investidores que buscam identificar oportunidades e riscos de mercado de forma eficiente.


---

#1. Bibliotecas

In [ ]:
!pip install mplfinance

In [ ]:
import pandas as pd
import numpy as np
import pytz
import yfinance as yf
import mplfinance as mpf
import plotly.graph_objects as go

#2. Obtenção de dados OHLC

In [ ]:
lista_tickers = ['ABEV3', 'ALPA4', 'ALSO3', 'ARZZ3', 'ASAI3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4',
                 'BBSE3', 'BEEF3', 'BPAC11', 'BPAN4', 'BRAP4', 'BRFS3', 'BRKM5', 'CASH3', 'CCRO3', 'CIEL3',
                 'CMIG4', 'CMIN3', 'COGN3', 'CPFE3', 'CPLE6', 'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3',
                 'DXCO3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3', 'ENBR3', 'ENEV3', 'ENGI11', 'EQTL3',
                 'EZTC3', 'FLRY3', 'GGBR4', 'GOAU4', 'GOLL4', 'HAPV3', 'HYPE3', 'IGTI11', 'ITSA4', 'ITUB4',
                 'JBSS3', 'KLBN11', 'LREN3', 'LWSA3', 'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3',
                 'PETR3', 'PETR4', 'PETZ3', 'PRIO3', 'QUAL3', 'RADL3', 'RAIL3', 'RAIZ4', 'RDOR3', 'RENT3',
                 'RRRP3', 'SANB11', 'SBSP3', 'SLC3', 'SMTO3', 'SOMA3', 'SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
                 'UGPA3', 'USIM5', 'VALE3', 'VBBR3', 'VIIA3', 'VIVT3', 'WEGE3', 'YDUQ3']

lista_tickers_yf = [x + '.SA' for x in lista_tickers]

In [ ]:
# Lista para armazenar tickers com falha
tickers_remover = []
indice_ativo = 0
ls_close_ativos = []
falha_download_ativos = []

for i in lista_tickers_yf:
    try:
        # Tenta fazer o download dos dados
        df_1d_ativo = yf.download(i, period="1y", interval='1d', auto_adjust=True)
        if df_1d_ativo.empty:
            raise ValueError(f"DataFrame vazio para o ticker {i}")

        df_1d_ativo['Ticker'] = i
        df_1d_ativo['Indice_ativo'] = indice_ativo
        indice_ativo += 1
        ls_close_ativos.append(df_1d_ativo)
    except Exception as e:
        print(f"Erro ao baixar dados para o ticker {i}: {e}")
        falha_download_ativos.append(i)
        tickers_remover.append(i)

# Atualiza lista_tickers_yf removendo tickers com falha
lista_tickers_yf = [ticker for ticker in lista_tickers_yf if ticker not in tickers_remover]

list(enumerate(lista_tickers_yf))
pass

#3. Construção das Funções

###Função que vai Localizar Topos e Fundos

In [ ]:
def localizador_fundos(df, n_candles=5):
  df_invertido = df.iloc[::-1]

  cond_low_anteriores = df['Low'] <= df['Low'].rolling(n_candles, closed='left').min()
  cond_low_posteriores = df_invertido['Low'] <= df_invertido['Low'].rolling(n_candles, closed='left').min()

  return (cond_low_anteriores & cond_low_posteriores)

def localizador_topos(df, n_candles=5):
  df_invertido = df.iloc[::-1]

  cond_high_anteriores = df['High'] <= df['High'].rolling(n_candles, closed='left').max()
  cond_high_posteriores = df_invertido['High'] <= df_invertido['High'].rolling(n_candles, closed='left').max()

  return (cond_high_anteriores & cond_high_posteriores)

Percorrendo em todos os ativos as funções de localizar topos e fundos mais atuais e criando uma lista com todos os dataframes que contem as informações de topos e fundos de todos os ativos

In [ ]:
n_candles = 10 # esse valor que indica quantos candles passados e futuros o algoritmo vai avaliar para indicar um topo ou fundo.
#exemplo: n_candles = 10 significa que um topo só será valido se ele for maior que os ultimos 10 candles e os 10 candles a frente.

ls_ativos_topos_fundos = []


for df in ls_close_ativos:
  print(f'Rodando para o ticker {df.Ticker.iloc[0]}...')


  ls_vertices = ['neutro']
  dict_topos_fundos = {'vertice': [], 'data': [], 'preco': []}

  for i in range(len(df)):

    if localizador_fundos(df, n_candles).iloc[i]:

      if ls_vertices[-1] != 'fundo':
        dict_topos_fundos['vertice'].append('fundo')
        dict_topos_fundos['data'].append(df.index[i])
        dict_topos_fundos['preco'].append(df.Low.iloc[i])
        ls_vertices.append('fundo')

      else:
        if df.Low.iloc[i] < dict_topos_fundos['preco'][-1]:
          dict_topos_fundos['data'][-1] = df.index[i]
          dict_topos_fundos['preco'][-1] = df.Low.iloc[i]

    if localizador_topos(df, n_candles).iloc[i]:

      if ls_vertices[-1] != 'topo':
        dict_topos_fundos['vertice'].append('topo')
        dict_topos_fundos['data'].append(df.index[i])
        dict_topos_fundos['preco'].append(df.High.iloc[i])
        ls_vertices.append('topo')

      else:
        if df.High.iloc[i] > dict_topos_fundos['preco'][-1]:
          dict_topos_fundos['data'][-1] = df.index[i]
          dict_topos_fundos['preco'][-1] = df.High.iloc[i]


  df_topos_fundos = pd.DataFrame(dict_topos_fundos)
  df_topos_fundos.index = df_topos_fundos.data

  df_final = pd.merge(df, df_topos_fundos, left_index=True, right_index=True, how='left')

  ls_ativos_topos_fundos.append(df_final)


Função que vai verificar se o preço atual está rompendo o topo ou fundo atual.

In [ ]:
def ativo_rompeu_topo(df):
    # Filtra o DataFrame para obter a última linha onde o vertice é 'topo'
    topo_df = df[df.vertice == 'topo']

    if topo_df.empty:
        return 'Nenhum topo encontrado'

    ultimo_topo = topo_df['preco'].iloc[-1]

    preco_atual = df['Close'].iloc[-1]

    # Calcula a distância do topo em porcentagem
    distancia_topo = ((ultimo_topo - preco_atual) / ultimo_topo) * 100

    if distancia_topo < 0:
        return 'rompeu topo'
    else:
        return 'não rompeu topo'

def ativo_rompeu_fundo(df):
    # Filtra o DataFrame para obter a última linha onde o vertice é 'fundo'
    fundo_df = df[df.vertice == 'fundo']

    if fundo_df.empty:
        return 'Nenhum fundo encontrado'

    ultimo_fundo = fundo_df['preco'].iloc[-1]

    preco_atual = df['Close'].iloc[-1]

    # Calcula a distância do fundo em porcentagem
    distancia_fundo = ((preco_atual - ultimo_fundo) / ultimo_fundo) * 100

    if distancia_fundo < 0:
        return 'rompeu fundo'
    else:
        return 'não rompeu fundo'



#Chamando função e executando o Screen

In [ ]:
for df in ls_ativos_topos_fundos:
    topo_status = ativo_rompeu_topo(df)
    fundo_status = ativo_rompeu_fundo(df)

    ticker = df.Ticker.iloc[0]
    indice_ativo = df.Indice_ativo.iloc[0]

    if topo_status == 'rompeu topo':
        print(f'🟢{ticker} Rompeu topo anterior🟢')

    if fundo_status == 'rompeu fundo':
        print(f'🔴{ticker} Rompeu fundo anterior🔴')